In [1]:
from pathlib import Path
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.pyplot as plt
import numpy as np
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %matplotlib inline
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
from mpl_toolkits.axes_grid1 import AxesGrid, ImageGrid
import pandas as pd 
import geopandas as gpd
import pygeos
import seaborn as sns
import xarray as xr
import rioxarray as rio
from matplotlib.colors import LinearSegmentedColormap
import datetime
from datetime import timedelta
from scipy.spatial.distance import cdist
from math import dist
from scipy.stats import ttest_ind
from geopy import distance
from scipy import stats
from matplotlib.gridspec import GridSpec
from pylr2 import regress2
from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)
from matplotlib.ticker import MaxNLocator,FixedLocator
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.stats import gaussian_kde

# import shutil

/Users/weitao/anaconda3/lib/python3.10/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Get hour Pandora

In [2]:
lisPD = []
# total VCD
for path in Path('./fus5/').rglob('*fus*.txt'):
    lisPD.append(path)
print(len(lisPD))
lisPD[:5]

19


[PosixPath('fus5/Pandora149s1_Seoul-SNU_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora54s1_Seoul_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora77s1_Singapore-NUS_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora217s1_Dalanzadgad_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora20s1_Busan_L2_rfus5p1-8.txt')]

In [4]:
pandora = pd.DataFrame(columns = ['Time', 'L1QA', 'L2fitQA', 'L2HCHOQA', 'VCD(moles/m2)', 'Status',
       'Location', 'Country', 'Lat', 'Lon', 'Altitude', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'Datetime', 'Localtime'])

for i in lisPD:
    cols = list(range(1,53))

    # Read txt to df
    PD = pd.read_csv(i,skiprows=74, encoding='unicode_escape',delimiter=' ',header=0,names=cols)
    PD = PD.iloc[:, [0,29,32,38,42,43,50,35]]

    cols2 = ['Time','L1QA','L2fitQA','VCD(moles/m2)','TotalUncertainty','RmsUncertainty','L1Height','L2HCHOQA']

    PD.columns = cols2
    # Read Attributes
    file = open(i, encoding='unicode_escape')
    content = file.readlines()
    status = content[4][43:-1]
    location = content[13][21:-1]
    country = content[14][21:-1]
    lat = float(content[15][25:-1])
    lon = float(content[16][26:-1])
    altitude = float(content[17][23:-1])
    
    timezone = int(round(lon/15))
    
    PD['Status'] = status
    PD['Location'] = location
    PD['Country'] = country
    PD['Lat'] = lat
    PD['Lon'] = lon
    PD['Altitude'] = altitude
    
    yearLis = []
    monthLis = []
    dayLis = []
    hourLis = []
    minuteLis = []
    datetimeLis = []
    for j in PD['Time']:
        yearLis.append(j[:4])
        monthLis.append(j[4:6])
        dayLis.append(j[6:8])
        hourLis.append(j[9:11])
        minuteLis.append(j[11:13])
        datetime_obj = datetime.datetime(int(j[:4]), int(j[4:6]), int(j[6:8]), int(j[9:11]), int(j[11:13]))
        datetimeLis.append(datetime_obj)
    PD['Year'] = yearLis
    PD['Month'] = monthLis
    PD['Day'] = dayLis
    PD['Hour'] = hourLis
    PD['Minute'] = minuteLis
    PD['Datetime'] = datetimeLis
    PD['Localtime'] = PD['Datetime'] + pd.Timedelta(hours=timezone)
    pandora = pd.concat([pandora,PD])
pandora

,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,...,Year,Month,Day,Hour,Minute,Datetime,Localtime,TotalUncertainty,RmsUncertainty,L1Height
0,20190207T023554.1Z,10,12,22,-0.001059,official,Seoul-SNU,South Korea,37.4580,126.951,...,2019,02,07,02,35,2019-02-07 02:35:00,2019-02-07 10:35:00,0.000080,0.000242,0.000
1,20190207T023727.4Z,10,12,22,-0.001090,official,Seoul-SNU,South Korea,37.4580,126.951,...,2019,02,07,02,37,2019-02-07 02:37:00,2019-02-07 10:37:00,0.000080,0.000244,0.000
2,20190207T023857.9Z,10,12,22,-0.001040,official,Seoul-SNU,South Korea,37.4580,126.951,...,2019,02,07,02,38,2019-02-07 02:38:00,2019-02-07 10:38:00,0.000079,0.000247,0.000
3,20190207T024026.0Z,10,12,22,-0.001010,official,Seoul-SNU,South Korea,37.4580,126.951,...,2019,02,07,02,40,2019-02-07 02:40:00,2019-02-07 10:40:00,0.000080,0.000247,0.000
4,20190207T024154.9Z,10,12,22,-0.001013,official,Seoul-SNU,South Korea,37.4580,126.951,...,2019,02,07,02,41,2019-02-07 02:41:00,2019-02-07 10:41:00,0.000079,0.000251,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60798,20240213T065347.1Z,10,11,11,0.000108,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,2024,02,13,06,53,2024-02-13 06:53:00,2024-02-13 15:53:00,0.000008,0.000018,0.004
60799,20240213T070424.2Z,10,11,11,0.000116,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,2024,02,13,07,04,2024-02-13 07:04:00,2024-02-13 16:04:00,0.000008,0.000017,0.007
60800,20240213T071418.1Z,10,11,11,0.000119,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,2024,02,13,07,14,2024-02-13 07:14:00,2024-02-13 16:14:00,0.000008,0.000018,0.010
60801,20240213T073216.1Z,10,12,12,0.000116,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,2024,02,13,07,32,2024-02-13 07:32:00,2024-02-13 16:32:00,0.000011,0.000029,0.029


In [5]:
pandora = pandora[pandora['Year'].isin(['2021','2022','2023'])]
print(len(pandora))
pandora.head()

793864


,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,...,Year,Month,Day,Hour,Minute,Datetime,Localtime,TotalUncertainty,RmsUncertainty,L1Height
38185,20210101T000840.4Z,12,12,12,0.000134,official,Seoul-SNU,South Korea,37.458,126.951,...,2021,01,01,00,08,2021-01-01 00:08:00,2021-01-01 08:08:00,0.000054,0.000073,0.007
38186,20210101T001442.7Z,10,11,12,0.000140,official,Seoul-SNU,South Korea,37.458,126.951,...,2021,01,01,00,14,2021-01-01 00:14:00,2021-01-01 08:14:00,0.000016,0.000018,0.006
38187,20210101T002425.7Z,10,11,12,0.000145,official,Seoul-SNU,South Korea,37.458,126.951,...,2021,01,01,00,24,2021-01-01 00:24:00,2021-01-01 08:24:00,0.000017,0.000019,0.004
38188,20210101T003434.7Z,10,11,12,0.000127,official,Seoul-SNU,South Korea,37.458,126.951,...,2021,01,01,00,34,2021-01-01 00:34:00,2021-01-01 08:34:00,0.000019,0.000018,0.003
38189,20210101T004045.3Z,12,12,12,0.000181,official,Seoul-SNU,South Korea,37.458,126.951,...,2021,01,01,00,40,2021-01-01 00:40:00,2021-01-01 08:40:00,0.000052,0.000055,0.003


In [6]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))
pandora['Datetime1']=pandora['Datetime'].map(hour_rounder)
pandora.head()

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_69661/33972301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora['Datetime1']=pandora['Datetime'].map(hour_rounder)


,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,...,Month,Day,Hour,Minute,Datetime,Localtime,TotalUncertainty,RmsUncertainty,L1Height,Datetime1
38185,20210101T000840.4Z,12,12,12,0.000134,official,Seoul-SNU,South Korea,37.458,126.951,...,01,01,00,08,2021-01-01 00:08:00,2021-01-01 08:08:00,0.000054,0.000073,0.007,2021-01-01 00:00:00
38186,20210101T001442.7Z,10,11,12,0.000140,official,Seoul-SNU,South Korea,37.458,126.951,...,01,01,00,14,2021-01-01 00:14:00,2021-01-01 08:14:00,0.000016,0.000018,0.006,2021-01-01 00:00:00
38187,20210101T002425.7Z,10,11,12,0.000145,official,Seoul-SNU,South Korea,37.458,126.951,...,01,01,00,24,2021-01-01 00:24:00,2021-01-01 08:24:00,0.000017,0.000019,0.004,2021-01-01 00:00:00
38188,20210101T003434.7Z,10,11,12,0.000127,official,Seoul-SNU,South Korea,37.458,126.951,...,01,01,00,34,2021-01-01 00:34:00,2021-01-01 08:34:00,0.000019,0.000018,0.003,2021-01-01 01:00:00
38189,20210101T004045.3Z,12,12,12,0.000181,official,Seoul-SNU,South Korea,37.458,126.951,...,01,01,00,40,2021-01-01 00:40:00,2021-01-01 08:40:00,0.000052,0.000055,0.003,2021-01-01 01:00:00


In [37]:
# Thus, we use "10", which is "unassured high quality"
pandora2 = pandora
pandora2 = pandora2[pandora2['L1QA'].isin([0,10])]
pandora2 = pandora2[pandora2['L2fitQA'].isin([0,10])]
pre = pandora2['Location'].value_counts()
pre
# pandora2 = pandora2[~pandora2['L2HCHOQA'].isin([20,21,22])]
# pandora2[pandora2['Location']=='Bangkok']

Seosan               39848
Seoul-SNU            38958
Tsukuba-NIES         36902
Dalanzadgad          32704
Bangkok              32046
Busan                29648
Tokyo-TMU            23979
Tsukuba              22241
Incheon-ESC          21584
Seoul                20486
Fukuoka              19126
Nagoya               18007
Yokosuka             16786
Beijing-RADI         15988
Dhaka                14141
Agam                 13325
Singapore-NUS        11083
Tsukuba-NIES-West     9326
Kobe                  9263
Name: Location, dtype: int64

In [38]:

# pandora2 = pandora2[~pandora2['L2HCHOQA'].isin([20, 21, 22, 12, 2])]
pandora2 = pandora2[pandora2['L2HCHOQA'].isin([0, 1, 10, 11])]

pos = pandora2['Location'].value_counts()
pos

Tsukuba-NIES         25851
Seoul-SNU            21608
Seosan               20760
Tokyo-TMU            17337
Dalanzadgad          17188
Busan                15558
Tsukuba              14198
Fukuoka              11913
Incheon-ESC          11233
Seoul                10634
Beijing-RADI         10438
Nagoya               10355
Yokosuka              9918
Agam                  5330
Kobe                  4780
Tsukuba-NIES-West     4758
Singapore-NUS         2772
Dhaka                 1983
Bangkok                432
Name: Location, dtype: int64

In [47]:
pandora2['L2HCHOQA'].value_counts()

10    141178
0      59101
11     12494
1       4273
Name: L2HCHOQA, dtype: int64

In [36]:
df1 = pd.DataFrame({'location':pre.index, 'pre':pre.values}).sort_values('location').reset_index()
df2 = pd.DataFrame({'location':pos.index, 'counts':pos.values}).sort_values('location').reset_index()
df1['post'] = df2['counts']
df1['delta'] = df1['pre'] - df1['post']
df1['ratio'] = df1['post'] / df1['pre']
df1

,index,location,pre,post,delta,ratio
0,15,Agam,13325,5330,7995,0.400000
1,4,Bangkok,32046,432,31614,0.013481
2,13,Beijing-RADI,15988,10438,5550,0.652865
3,5,Busan,29648,15558,14090,0.524757
4,3,Dalanzadgad,32704,17188,15516,0.525563
5,14,Dhaka,14141,1983,12158,0.140231
6,10,Fukuoka,19126,11913,7213,0.622869
7,8,Incheon-ESC,21584,11233,10351,0.520432
8,18,Kobe,9263,4780,4483,0.516032
9,11,Nagoya,18007,10355,7652,0.575054


In [39]:
# We only use data that is close to GEMS observation time
pandora2['Minute'] = pandora2['Minute'].astype(int)
pandora3 = pandora2[(pandora2['Minute']<=15)|(pandora2['Minute']>=45)].copy()
pandora3

,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,...,Month,Day,Hour,Minute,Datetime,Localtime,TotalUncertainty,RmsUncertainty,L1Height,Datetime1
38314,20210103T020311.0Z,10,10,10,0.000186,official,Seoul-SNU,South Korea,37.4580,126.951,...,01,03,02,3,2021-01-03 02:03:00,2021-01-03 10:03:00,0.000026,0.000020,0.001,2021-01-03 02:00:00
38478,20210106T020638.3Z,10,10,10,0.000173,official,Seoul-SNU,South Korea,37.4580,126.951,...,01,06,02,6,2021-01-06 02:06:00,2021-01-06 10:06:00,0.000027,0.000020,0.001,2021-01-06 02:00:00
38479,20210106T021200.2Z,10,10,10,0.000172,official,Seoul-SNU,South Korea,37.4580,126.951,...,01,06,02,12,2021-01-06 02:12:00,2021-01-06 10:12:00,0.000027,0.000020,0.001,2021-01-06 02:00:00
38485,20210106T024835.3Z,10,10,10,0.000178,official,Seoul-SNU,South Korea,37.4580,126.951,...,01,06,02,48,2021-01-06 02:48:00,2021-01-06 10:48:00,0.000029,0.000021,0.001,2021-01-06 03:00:00
38486,20210106T025817.3Z,10,10,10,0.000177,official,Seoul-SNU,South Korea,37.4580,126.951,...,01,06,02,58,2021-01-06 02:58:00,2021-01-06 10:58:00,0.000030,0.000021,0.001,2021-01-06 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58325,20231230T030925.5Z,10,10,10,0.000131,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,12,30,03,9,2023-12-30 03:09:00,2023-12-30 12:09:00,0.000009,0.000022,0.000,2023-12-30 03:00:00
58326,20231230T031445.8Z,10,10,10,0.000130,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,12,30,03,14,2023-12-30 03:14:00,2023-12-30 12:14:00,0.000008,0.000023,0.000,2023-12-30 03:00:00
58332,20231230T035210.4Z,10,10,10,0.000151,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,12,30,03,52,2023-12-30 03:52:00,2023-12-30 12:52:00,0.000008,0.000023,0.001,2023-12-30 04:00:00
58334,20231230T040842.9Z,10,10,10,0.000159,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,12,30,04,8,2023-12-30 04:08:00,2023-12-30 13:08:00,0.000008,0.000022,0.001,2023-12-30 04:00:00


In [40]:
# pandora3 = pandora3[[  'VCD(moles/m2)',  'Location', 
#        'Lat', 'Lon', 'Datetime1']]
pandora3['Year'] = pandora3['Datetime1'].dt.year
pandora3['Month'] = pandora3['Datetime1'].dt.month
pandora3['Day'] = pandora3['Datetime1'].dt.day
pandora3['Hour'] = pandora3['Datetime1'].dt.hour

pandora3['time']= pandora3['Hour']+pandora3['Day']*100+pandora3['Month']*10000+pandora3['Year']*1000000
pandora3['time']= pandora3['time'].astype(int)
pandora3['time']= pandora3['time'].astype(str)

pandora3['ID'] = pandora3['time']+' '+pandora3['Location']

pandora3

,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,...,Hour,Minute,Datetime,Localtime,TotalUncertainty,RmsUncertainty,L1Height,Datetime1,time,ID
38314,20210103T020311.0Z,10,10,10,0.000186,official,Seoul-SNU,South Korea,37.4580,126.951,...,2,3,2021-01-03 02:03:00,2021-01-03 10:03:00,0.000026,0.000020,0.001,2021-01-03 02:00:00,2021010302,2021010302 Seoul-SNU
38478,20210106T020638.3Z,10,10,10,0.000173,official,Seoul-SNU,South Korea,37.4580,126.951,...,2,6,2021-01-06 02:06:00,2021-01-06 10:06:00,0.000027,0.000020,0.001,2021-01-06 02:00:00,2021010602,2021010602 Seoul-SNU
38479,20210106T021200.2Z,10,10,10,0.000172,official,Seoul-SNU,South Korea,37.4580,126.951,...,2,12,2021-01-06 02:12:00,2021-01-06 10:12:00,0.000027,0.000020,0.001,2021-01-06 02:00:00,2021010602,2021010602 Seoul-SNU
38485,20210106T024835.3Z,10,10,10,0.000178,official,Seoul-SNU,South Korea,37.4580,126.951,...,3,48,2021-01-06 02:48:00,2021-01-06 10:48:00,0.000029,0.000021,0.001,2021-01-06 03:00:00,2021010603,2021010603 Seoul-SNU
38486,20210106T025817.3Z,10,10,10,0.000177,official,Seoul-SNU,South Korea,37.4580,126.951,...,3,58,2021-01-06 02:58:00,2021-01-06 10:58:00,0.000030,0.000021,0.001,2021-01-06 03:00:00,2021010603,2021010603 Seoul-SNU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58325,20231230T030925.5Z,10,10,10,0.000131,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,3,9,2023-12-30 03:09:00,2023-12-30 12:09:00,0.000009,0.000022,0.000,2023-12-30 03:00:00,2023123003,2023123003 Tsukuba-NIES
58326,20231230T031445.8Z,10,10,10,0.000130,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,3,14,2023-12-30 03:14:00,2023-12-30 12:14:00,0.000008,0.000023,0.000,2023-12-30 03:00:00,2023123003,2023123003 Tsukuba-NIES
58332,20231230T035210.4Z,10,10,10,0.000151,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,4,52,2023-12-30 03:52:00,2023-12-30 12:52:00,0.000008,0.000023,0.001,2023-12-30 04:00:00,2023123004,2023123004 Tsukuba-NIES
58334,20231230T040842.9Z,10,10,10,0.000159,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,...,4,8,2023-12-30 04:08:00,2023-12-30 13:08:00,0.000008,0.000022,0.001,2023-12-30 04:00:00,2023123004,2023123004 Tsukuba-NIES


In [44]:
# pandora3['L2HCHOQA'] = pandora3['L2HCHOQA'].astype(int)
pandora4 = pandora3.groupby(['ID']).median()
# pandora4['time']= pandora4['Hour']+pandora4['Day']*100+pandora4['Month']*10000+pandora4['Year']*1000000
# pandora4['time']= pandora4['time'].astype(int)
# pandora4['time']= pandora4['time'].astype(str)
pandora4 = pandora4.reset_index()
pandora4[['time', 'loca']] = pandora4['ID'].str.split(' ', n=1, expand=True)

datetimeLis=[]
for j in pandora4['time']:

        datetime_obj = datetime.datetime(int(j[:4]), int(j[4:6]), int(j[6:8]), int(j[8:10]))
        datetime_obj = datetime_obj - timedelta(minutes=15)
        datetimeLis.append(datetime_obj)
pandora4['datetime']=datetimeLis
pandora4['Year'] = pandora4['datetime'].dt.year
pandora4['Month'] = pandora4['datetime'].dt.month
pandora4['Day'] = pandora4['datetime'].dt.day
pandora4['Hour'] = pandora4['datetime'].dt.hour
pandora4['time']= pandora4['Hour']+pandora4['Day']*100+pandora4['Month']*10000+pandora4['Year']*1000000

# pandora4 = pandora4[[ 'VCD(moles/m2)', 'Lat', 'Lon', 'Hour',
#        'time', 'loca', 'datetime']]
pandora4 = pandora4[pandora4['Hour'].isin([0,1,2,3,4,5,6,7,22,23])]
pandora4

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_69661/438308052.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pandora4 = pandora3.groupby(['ID']).median()


,ID,VCD(moles/m2),Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,TotalUncertainty,RmsUncertainty,L1Height,time,loca,datetime
0,2021010104 Yokosuka,0.000041,35.3207,139.6508,5.0,2021,1,1,3,50.0,0.000008,0.000023,0.0005,2021010103,Yokosuka,2021-01-01 03:45:00
1,2021010201 Seosan,0.000087,36.7769,126.4938,25.0,2021,1,2,0,32.5,0.000008,0.000008,0.0020,2021010200,Seosan,2021-01-02 00:45:00
2,2021010202 Seosan,0.000092,36.7769,126.4938,25.0,2021,1,2,1,12.5,0.000009,0.000007,0.0010,2021010201,Seosan,2021-01-02 01:45:00
3,2021010203 Seosan,0.000070,36.7769,126.4938,25.0,2021,1,2,2,46.0,0.000009,0.000008,0.0000,2021010202,Seosan,2021-01-02 02:45:00
4,2021010203 Yokosuka,0.000047,35.3207,139.6508,5.0,2021,1,2,2,13.0,0.000008,0.000023,0.0000,2021010202,Yokosuka,2021-01-02 02:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33098,2023123005 Tokyo-TMU,0.000195,35.6200,139.3834,135.0,2023,12,30,4,34.0,0.000010,0.000018,0.0010,2023123004,Tokyo-TMU,2023-12-30 04:45:00
33099,2023123007 Dhaka,0.000467,23.7284,90.3982,34.0,2023,12,30,6,48.0,0.000040,0.000026,0.0000,2023123006,Dhaka,2023-12-30 06:45:00
33101,2023123102 Tokyo-TMU,0.000173,35.6200,139.3834,135.0,2023,12,31,1,5.0,0.000011,0.000021,0.0000,2023123101,Tokyo-TMU,2023-12-31 01:45:00
33102,2023123103 Tokyo-TMU,0.000184,35.6200,139.3834,135.0,2023,12,31,2,52.0,0.000011,0.000020,0.0000,2023123102,Tokyo-TMU,2023-12-31 02:45:00


In [42]:
pandora4 = pandora4.sort_values('time')

In [43]:
pandora4.to_pickle('Pandora_fus5_DS2')

In [46]:
pd.read_pickle('Pandora_fus5_DS')

,ID,VCD(moles/m2),Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,TotalUncertainty,RmsUncertainty,L1Height,time,loca,datetime
0,2021010103 Yokosuka,0.000029,35.3207,139.6508,5.0,2021,1,1,2,5.0,0.000008,0.000018,0.0000,2021010102,Yokosuka,2021-01-01 02:45:00
1,2021010104 Seoul-SNU,0.000231,37.4580,126.9510,116.0,2021,1,1,3,52.0,0.000030,0.000023,0.0000,2021010103,Seoul-SNU,2021-01-01 03:45:00
2,2021010104 Yokosuka,0.000041,35.3207,139.6508,5.0,2021,1,1,3,50.0,0.000008,0.000023,0.0005,2021010103,Yokosuka,2021-01-01 03:45:00
3,2021010105 Yokosuka,0.000006,35.3207,139.6508,5.0,2021,1,1,4,55.5,0.000008,0.000019,0.0010,2021010104,Yokosuka,2021-01-01 04:45:00
4,2021010201 Seosan,0.000086,36.7769,126.4938,25.0,2021,1,2,0,49.5,0.000008,0.000008,0.0020,2021010200,Seosan,2021-01-02 00:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52932,2023123106 Dhaka,0.000283,23.7284,90.3982,34.0,2023,12,31,5,8.5,0.000041,0.000024,0.0000,2023123105,Dhaka,2023-12-31 05:45:00
52933,2023123107 Bangkok,0.000462,13.7847,100.5400,60.0,2023,12,31,6,12.5,0.000016,0.000030,0.0000,2023123106,Bangkok,2023-12-31 06:45:00
52934,2023123107 Dhaka,0.000300,23.7284,90.3982,34.0,2023,12,31,6,11.0,0.000039,0.000023,0.0000,2023123106,Dhaka,2023-12-31 06:45:00
52935,2023123108 Bangkok,0.000465,13.7847,100.5400,60.0,2023,12,31,7,15.0,0.000016,0.000024,0.0000,2023123107,Bangkok,2023-12-31 07:45:00
